#### 1. import the required modules

In [29]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#### 2. load up the breast cancer dataset

In [34]:
dataset = datasets.load_breast_cancer()
X = dataset.data
y = dataset.target

#### 3. get the number of samples and number of features

In [35]:
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


#### 4. split the dataset into train and test datasets

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

#### 5. scale the dataset

In [37]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### 6. convert data from np array to pytorch tensors

In [38]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

#### 7. reshape the y tensors to column vectors

In [39]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

#### 8. setup the model class

In [40]:
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

#### 9. instantiate the model

In [41]:
model = LogisticRegression(n_features)

#### 10. create the loss function and optimizer

In [42]:
learning_rate = 0.01
loss_function =  nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#### 11. peform the training in 100 epochs

In [43]:
epochs = 100
for epoch in range(epochs):
    
    # forward pass
    y_predicted = model(X_train)
    loss = loss_function(y_predicted, y_train)
    
    # backward pass
    loss.backward()
    
    # weight updates
    optimizer.step()
    
    # empty the gradients
    optimizer.zero_grad()
    
    # print the losses
    if (epoch + 1) % 10 == 0:
        print(f"epoch: {epoch+1}, loss: {loss.item():.5f}")
    

epoch: 10, loss: 0.54402
epoch: 20, loss: 0.45974
epoch: 30, loss: 0.40289
epoch: 40, loss: 0.36188
epoch: 50, loss: 0.33076
epoch: 60, loss: 0.30622
epoch: 70, loss: 0.28630
epoch: 80, loss: 0.26975
epoch: 90, loss: 0.25573
epoch: 100, loss: 0.24367


#### 12. evaluate the model accuracy using the test data

In [45]:
y_predicted = model(X_test)
y_predicted_class = y_predicted.round()

accuracy = y_predicted_class.eq(y_test).sum() / float(y_test.shape[0])
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9035
